
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Lab - Monitoring Drift with Lakehouse Monitoring

In this lab, you will investigate how to monitor drift. You will work through a series of practical exercises to simulate prediction drift, set up monitoring for inference tables, and analyze drift metrics. The lab also includes hands-on tasks to explore visual dashboards, extract and modify SQL code, and create custom alerts. Additionally, you will learn to inspect lineage, enabling you to trace the flow of data and associated dashboards and queries.


**Learning Objectives**
By the end of this lab, you will have completed the following:
- Initiate Lakehouse Monitoring programmatically
- Inspect automatically generated Dashboard
- Create a new widget in the monitoring dashboard
- Inspect Profile and Drift Metrics tables
- Setup an Alert

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.
Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.
1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
   - In the drop-down, select **More**.
   - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
1. Find the triangle icon to the right of your compute cluster name and click it.
1. Wait a few minutes for the cluster to start.
1. Once the cluster is running, complete the steps above to select your cluster.


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **17.3.x-cpu-ml-scala2.13**

* To utilize the generated Dashboard, you will need a SQL Warehouse.

## Classroom Setup

To get into the lesson, we first need to build some data assets and define some configuration variables required for this lab. When running the following cell, the output is hidden so our space isn't cluttered. To view the details of the output, you can hover over the next cell and click the eye icon. 

The cell after the setup, titled `View Setup Variables`, displays the various variables that were created. You can click the Catalog icon in the notebook space to the right to see that your catalog was created with no data.

In [0]:
%run ../Includes/Classroom-Setup-3.Lab

## Initiate Inference Monitoring

Here you will use Python to initiate the monitoring of your inference table using Lakehouse Monitoring. This will generate two table: profile metrics and drift metrics. In addition, a Databricks Dashboard will also be generated. We will inspect all these assets later. 

Terminology: 
- Your **primary table** is your inference table you created by running the classroom setup. The name of this table is `model_logs`.
- Your **baseline table** is your source of truth for your data. The name of this table is `baseline_features`

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorInferenceLog, MonitorInferenceLogProblemType, MonitorInfoStatus, MonitorRefreshInfoState, MonitorMetric
w = WorkspaceClient()
full_primary_table_name = <FILL_IN>
full_baseline_table_name = <FILL_IN>

In [0]:
%skip

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorInferenceLog, MonitorInferenceLogProblemType, MonitorInfoStatus, MonitorRefreshInfoState, MonitorMetric
w = WorkspaceClient()
full_primary_table_name = f'{DA.catalog_name}.{DA.schema_name}.model_logs'
full_baseline_table_name = f"{DA.catalog_name}.{DA.schema_name}.baseline_features"

During the initial classroom setup, we registered our inference table with Unity Catalog that contained features, predictions, and labels for a classification problem. Next, we will set essential variables that will be used for enabling Lakehouse Monitoring and initiate monitoring. 

Configure the setup with the following conditions. 
1. Set the window size to **1 day**
1. Store the generated notebook in a folder in your homeworkspace with the path `<username>/My_Monitored_Dashboards/Drift_Detection`
1. Set the slicer to include `Ages` between 3 and 10 and `HvyAlcoholConsump` to be 1 (non-alcoholic).

In [0]:
## ML problem type, either "classification" or "regression"
PROBLEM_TYPE = MonitorInferenceLogProblemType.<FILL_IN>

## Window sizes to analyze data over
GRANULARITIES = <FILL_IN>   

## Directory to store generated dashboard
ASSETS_DIR = f"/Workspace/Users/{DA.username}/databricks_lakehouse_monitoring/{primary_table_name}"

## Optional parameters
SLICING_EXPRS = <FILL_IN>   # Expressions to slice data with

In [0]:
%skip

## ML problem type, either "classification" or "regression"
PROBLEM_TYPE = MonitorInferenceLogProblemType.PROBLEM_TYPE_CLASSIFICATION

## Window sizes to analyze data over
GRANULARITIES = ["1 day"]   

## Directory to store generated dashboard
ASSETS_DIR = f"/Workspace/Users/{DA.username}/My_Monitored_Dashboards/Drift_Detection"

## Optional parameters
SLICING_EXPRS = ["Age < 10", "Age > 3", "HvyAlcoholConsump = 1"]   # Expressions to slice data with

Run the initialization for Lakehouse Monitoring using the configuration above. Recall that for this dataset, we are interested in `Diabetes_binary` as our target variable.

In [0]:
print(f"Creating monitor for model_logs")
try: 
  info = w.quality_monitors.create(
    table_name=<FILL_IN>,
    inference_log=MonitorInferenceLog(
      timestamp_col='timestamp',
      granularities=<FILL_IN>,
      model_id_col='model_id',  ## Model version number 
      prediction_col=<FILL_IN>,  ## Ensure this column is of type DOUBLE
      problem_type=<FILL_IN>,
      label_col=<FILL_IN>  ## Ensure this column is of type DOUBLE
    ),
    baseline_table_name=<FILL_IN>,
    slicing_exprs=<FILL_IN>,
    output_schema_name=f"{DA.catalog_name}.{DA.schema_name}",
    assets_dir=<FILL_IN>
  )

  import time

  ## Wait for monitor to be created
  while info.status ==  MonitorInfoStatus.MONITOR_STATUS_PENDING:
    info = w.quality_monitors.get(table_name=<FILL_IN>)
    time.sleep(10)

  assert info.status == MonitorInfoStatus.MONITOR_STATUS_ACTIVE, "Error creating monitor"
except Exception as e:
  print(f"Error creating monitor: {e}")

In [0]:
%skip
print(f"Creating monitor for model_logs")
try: 
  info = w.quality_monitors.create(
    table_name=full_primary_table_name,
    inference_log=MonitorInferenceLog(
      timestamp_col='timestamp',
      granularities=GRANULARITIES,
      model_id_col='model_id',  ## Model version number 
      prediction_col='Diabetes_binary',  ## Ensure this column is of type DOUBLE
      problem_type=PROBLEM_TYPE,
      label_col='labeled_data'  ## Ensure this column is of type DOUBLE
    ),
    baseline_table_name=full_baseline_table_name,
    slicing_exprs=SLICING_EXPRS,
    output_schema_name=f"{DA.catalog_name}.{DA.schema_name}",
    assets_dir=ASSETS_DIR
  )

  import time

  ## Wait for monitor to be created
  while info.status ==  MonitorInfoStatus.MONITOR_STATUS_PENDING:
    info = w.quality_monitors.get(table_name=full_primary_table_name)
    time.sleep(10)

  assert info.status == MonitorInfoStatus.MONITOR_STATUS_ACTIVE, "Error creating monitor"
except Exception as e:
  print(f"Error creating monitor: {e}")

In [0]:
## A metric refresh will automatically be triggered on creation
refreshes = w.quality_monitors.list_refreshes(table_name=<FILL_IN>).refreshes
assert(len(refreshes) > 0)

run_info = refreshes[0]
while run_info.state in (MonitorRefreshInfoState.PENDING, MonitorRefreshInfoState.RUNNING):
  run_info = w.quality_monitors.get_refresh(table_name=<FILL_IN>, refresh_id=run_info.refresh_id)
  print(run_info)
  time.sleep(30)

assert run_info.state == MonitorRefreshInfoState.SUCCESS, "Monitor refresh failed"

In [0]:
%skip
## A metric refresh will automatically be triggered on creation
refreshes = w.quality_monitors.list_refreshes(table_name=full_primary_table_name).refreshes
assert(len(refreshes) > 0)

run_info = refreshes[0]
while run_info.state in (MonitorRefreshInfoState.PENDING, MonitorRefreshInfoState.RUNNING):
  run_info = w.quality_monitors.get_refresh(table_name=full_primary_table_name, refresh_id=run_info.refresh_id)
  print(run_info)
  time.sleep(30)

assert run_info.state == MonitorRefreshInfoState.SUCCESS, "Monitor refresh failed"

## Locate the Monitoring Dashboard

There are two methods for locating the monitoring Dashboard.

1. Navigate to the underlying inference table `model_logs` and click on **Quality** and click on **View Dashboard**
1. Navigate to the underlying inference table `model_logs` and click on **Lineage** and click on **Open in a dashboard** and find the dashboard you just created.

### Inspect Monitoring Dashboard

1. After navigating to the canvas, scroll down until you see **Numerical Feature Drift**.
1. Click on the kebab on the widget with title **# Features with Numerical Drift**.
1. Click on **View Dataset**. This will take you to the generated **SQL code**. Modify it to not limit to the latest window by commenting out a single line.
1. Copy and paste your answer in the next cell.

**Do not run the SQL code** -  you will receive an error since you are working in a notebook. However, you can run the modified code in the Data tab of your dashboard.

In [0]:
## Copy and paste the code you modified above. 
<FILL_IN>

In [0]:
%skip

WITH 
profile_metrics AS (
  SELECT * FROM `dbacademy`.<`schema-name`>.`model_logs_profile_metrics`
  WHERE window.start >= :`Time Window Start` AND window.end <= :`Time Window End` -- limit to the inspection time window range
  AND isnull(slice_key) AND isnull(slice_value) -- default to "No Slice"
  AND   `model_id`   = "*" -- default to all model ids
),
last_window_in_inspection_range AS (
  SELECT window.start AS Window, granularity AS Granularity FROM profile_metrics
  WHERE window.start IN (SELECT MAX(window.start) FROM profile_metrics) 
  ORDER BY Granularity LIMIT 1 -- order to ensure the `granularity` selected is stable
),
profile_metrics_inspected AS (
  SELECT * FROM profile_metrics
  WHERE Granularity = (SELECT Granularity FROM last_window_in_inspection_range)
)
SELECT
  concat(window.start," - ", window.end) AS Window,
  CONCAT('"', ELEMENT_AT(frequent_items, 1).item, '", ', CAST((ELEMENT_AT(frequent_items, 1).count / count) * 100 AS INT), "%") AS top_class,
  granularity AS Granularity,
    `model_id`   AS `Model Id`,
  COALESCE(slice_key, "No slice") AS `Slice key`,
  COALESCE(slice_value, "No slice") AS `Slice value`
FROM profile_metrics_inspected
WHERE
  -- window.start IN (SELECT Window FROM last_window_in_inspection_range) -- limit to last window
  log_type = "INPUT"
  AND column_name = "Diabetes_binary"

> **⚠️ Note:**  
> You may encounter a `CAST_INVALID_INPUT` error in the **Numerical Feature Quantile Drift** widget.  
> This occurs because the auto-generated query attempts to cast the string `"Baseline"` to a `TIMESTAMP` type in the `Window` column.  
> While this error does not affect other metrics or drift detection functionality, you can optionally resolve it using the steps below.

**Steps to Diagnose and Fix the Error (Optional)**
- **Step 1:**  
  Navigate to the **Numerical Feature Quantile Drift** widget in the monitoring dashboard.  
  If there's a rendering failure, you will see a `CAST_INVALID_INPUT` error message displayed within the widget.
- **Step 2:**  
  Click the **kebab menu (⋮)** in the top-right corner of the widget and select **View Dataset**.  
  This opens the SQL query that powers the visual.
- **Step 3:**  
  In the dataset editor, scroll down and click **Diagnose error**.  
  The Databricks Assistant will analyze the issue and provide a suggested fix.
- **Step 4:**  
  Click **Replace active cell content** to apply the Assistant’s suggested query.  
  The corrected query will ensure the `Window` column in the baseline portion uses a proper timestamp type, typically by replacing `"Baseline"` with `CAST(NULL AS TIMESTAMP)`.
- **Step 5:**  
  Click **Run** to execute the updated query.
- **Step 6:**  
  Once the query runs successfully, the widget should render correctly, and the error message will disappear.
This fix helps ensure type consistency in the `UNION` clause across datasets, resolving the rendering error in the visual.

### Create a New Widget in the Canvas

Next, create a new widget that provides a visual counter for the precision macro during the last window. 

Steps: 
1. Navigate to the canvas you just created. 
1. Copy or create a new visualization using the **Add a visualization** blue button at the bottom of the Canvas screen. 
1. Set the Dataset value to **performance_last_window**. 
1. Set the **Visualization** to **Counter**.
1. Set the **Value** to **precision_macro**. 
1. Set the **Comparison** to **Window**.
1. Give it a title **Precision Macro** as the **Title** and **In the last time window** for the **Description**.

## Inspect Profile and Drift Metrics 

Now we will turn our focus to inspecting the profile and drift metric tables that were created when we set up Lakehouse Monitoring. Let's read in both tables.

In [0]:
## Display drift metrics table
drift_table = f"{DA.catalog_name}.{DA.schema_name}.<FILL_IN>"
drift_table_df = spark.sql(<FILL_IN>)
display(drift_table_df.orderBy(F.rand()))

## Display profile metrics table
profile_table = f"{DA.catalog_name}.{DA.schema_name}.<FILL_IN>"
profile_table_df = spark.sql(<FILL_IN>)
display(drift_table_df.orderBy(F.rand()))

In [0]:
%skip

## Display drift metrics table
drift_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_drift_metrics"
drift_table_df = spark.sql(f"SELECT * FROM {drift_table}")
display(drift_table_df.orderBy(F.rand()))

## Display profile metrics table
profile_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_profile_metrics"
profile_table_df = spark.sql(f"SELECT * FROM {profile_table}")
display(profile_table_df.orderBy(F.rand()))

### Use SQL to Analyze Profile Metrics Table

Next, display the profile metrics table where the baseline data was logged and the f1-score is not null and order by accuracy_score.

In [0]:
## Display profile metrics table
profile_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_profile_metrics"
profile_table_df = spark.sql(f"SELECT * FROM {profile_table} where f1_score is not null and log_type = 'BASELINE' order by accuracy_score")
display(profile_table_df.orderBy(F.rand()))

### Use SQL to inspect the drift metrics

Use SQL to inspect drift metrics by finding all records where
1. The drift type is baseline
1. No slice key
1. BMI data only

In [0]:
%sql
<FILL_IN>

In [0]:
%skip
%sql
select * from model_logs_drift_metrics
where drift_type = 'BASELINE' AND slice_key is null and column_name = 'BMI'

Build custom code to inspect drift using KS test.

In [0]:
import pandas as pd
import json

## Load the drift metrics data from the Delta table
drift_table =<FILL_IN>
drift_metrics_df = <FILL_IN>

## Convert to Pandas DataFrame
data = drift_metrics_df.toPandas()

## Convert Timestamp objects to strings in 'window' and 'window_cmp'
def convert_timestamp_to_string(d):
    if isinstance(d, dict):
        for k, v in d.items():
            if isinstance(v, pd.Timestamp):
                d[k] = v.isoformat()
            elif isinstance(v, dict):
                d[k] = convert_timestamp_to_string(v)
    return d

data['window'] = data['window'].apply(<FILL_IN>)
data['window_cmp'] = data['window_cmp'].apply(<FILL_IN>)

## Ensure JSON fields are strings
data['window'] = data['window'].apply(json.dumps)
data['window_cmp'] = data['window_cmp'].apply(json.dumps)

## Convert the JSON string in 'window' and 'window_cmp' to dictionaries
for index, row in data.iterrows():
    row['window'] = json.loads(row['window'])
    row['window_cmp'] = json.loads(row['window_cmp'])

## Analyze the drift metrics setting the threshold to 0.6
drift_thresholds = <FILL_IN>




def check_drift(row):
    ks_test_value = row['ks_test'].get('statistic') if isinstance(row['ks_test'], dict) else row['ks_test']
    if ks_test_value is not None and ks_test_value > drift_thresholds['ks_test']:
        return True
    return False

data['drift_detected'] = data.apply(<FILL_IN>, axis=1)

## Display rows with drift detected
drifted_rows = <FILL_IN>
no_drifted_rows = <FILL_IN>

print("Rows with drift detected:")
display(drifted_rows)

print("\nRows with no drift detected:")
display(no_drifted_rows)

In [0]:
%skip

import pandas as pd
import json

## Load the drift metrics data from the Delta table
drift_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_drift_metrics"
drift_metrics_df = spark.read.table(drift_table)

## Convert to Pandas DataFrame
data = drift_metrics_df.toPandas()

## Convert Timestamp objects to strings in 'window' and 'window_cmp'
def convert_timestamp_to_string(d):
    if isinstance(d, dict):
        for k, v in d.items():
            if isinstance(v, pd.Timestamp):
                d[k] = v.isoformat()
            elif isinstance(v, dict):
                d[k] = convert_timestamp_to_string(v)
    return d

data['window'] = data['window'].apply(convert_timestamp_to_string)
data['window_cmp'] = data['window_cmp'].apply(convert_timestamp_to_string)

## Ensure JSON fields are strings
data['window'] = data['window'].apply(json.dumps)
data['window_cmp'] = data['window_cmp'].apply(json.dumps)

## Convert the JSON string in 'window' and 'window_cmp' to dictionaries
for index, row in data.iterrows():
    row['window'] = json.loads(row['window'])
    row['window_cmp'] = json.loads(row['window_cmp'])

## Analyze the drift metrics
drift_thresholds = {
    "ks_test": 0.6,
}

def check_drift(row):
    ks_test_value = row['ks_test'].get('statistic') if isinstance(row['ks_test'], dict) else row['ks_test']
    if ks_test_value is not None and ks_test_value > drift_thresholds['ks_test']:
        return True
    return False

data['drift_detected'] = data.apply(check_drift, axis=1)

## Display rows with drift detected
drifted_rows = data[data['drift_detected']]
no_drifted_rows = data[~data['drift_detected']]

print("Rows with drift detected:")
display(drifted_rows)

print("\nRows with no drift detected:")
display(no_drifted_rows)


## Lineage and Alerts for Drift Detection

The following instructions are not meant to send you an alert to any device. It is only to demonstrate how to use lineage alongside alerts for your metric tables.

1. **Navigate to Queries**:
   - On the left-hand sidebar of the Databricks workspace, click on **Queries**.

1. **Create a Query**:
   - Click on **Create Query** in the top-right corner.
   - Write a query to calculate the average value of the **Count** column in your profile metrics table:

      ```SELECT AVG(Count) AS avg_count FROM model_logs_profile_metrics;```

   - Give the Query a name like `Drift Alert 1`
   > ****Note:**** Before running the query, make sure to set the current catalog and schema, and follow the `catalog.schema.table_name` structure.
   - Once the query is complete, click **Save** to save it.

1. **Create an Alert**:
   - Navigate to **Alerts** on the left-hand sidebar and click **Create Alert**.
   - Provide the alert with a **name** like `Test alert`.
   - In the SQL editor panel, paste the query you have saved before.
> ****Note:**** Before running the query, make sure to set the current catalog and schema, and follow the `catalog.schema.table_name` structure.
   - Click the **Run all** button to validate the query.
   - Define a **Trigger Condition**:
     - Example: Trigger the alert when the value is above 20.
   - Click **Test condition**.
     - If the result satisfies the condition, you’ll see:`Triggered: Alert will trigger based on current data.`
   - Under **Notify**, search and add users or destinations. 
   - Finally, click **View alert** and Set the **Schedule**.
1. **Monitor Lineage and Updates**:
   - Go back to the **Catalog** on the left sidebar.
   - Navigate to your **model_logs_profile_metrics** table.
   - Click on **Lineage** and then see the queries associated with this table.
      - You will see the `Drift Alert 1` listed there.
   - Whenever there’s an update or drift detected in your model or dataset, the alert will notify you automatically. You can then go to the table's **Lineage** tab to investigate the issue.

## Conclusion

In this lab, you learned how to programmatically setup Lakehouse Monitoring on an inference table for detecting drift. This included investigating the automatically generated dashboard as well as the two metric tables: profile metrics and drift metrics. Additionally, you set up an alert use SQL to notify you when you suspect drifting may occur.

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>